In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Load data

1. Get html content using `requests.get`
2. Parse the html using `BeautifulSoup`
3. Load the parsed table into pandas dataframe using `pd.read_html`
4. Drop `NaN` values in column Borough
5. Replace `NaN` values in column Neighbour with the value from Borough (ZERO case found for the dadtaset)
6. Merge two neighbourhoods with the same Postal Code (ZERO case found for the dataset)
7. Reorder the dataset

In [2]:
# auxiliary function to merge duplicated postal code that does not exist
def merge_duplicate(df):
    """
    Find all the rows in [df] with same 'Postal Code' value
        then join their 'Neighbourhood' values, inplace

    Example:
        >>> test = pd.DataFrame({ \
                'Postal Code': ('M1A', 'M1A', 'M2A', 'M3A', 'M1A', 'M2A'), \
                'Neighbourhood': ('A', 'B', 'C', 'D', 'E', 'F'), \
            })
        >>> merge_duplicate(test)
        >>> test.loc[test['Postal Code'] == 'M1A']['Neighbourhood'].values[0]
        'A, B, E'
        >>> test.loc[test['Postal Code'] == 'M2A']['Neighbourhood'].values[0]
        'C, F'
    """
    counts = df['Postal Code'].value_counts()
    duplicate = set([code for code, c in zip(counts.index, counts) if c > 1])
    
    for code in duplicate:
        indices = np.where(df['Postal Code'] == code)[0]
        neighbour_names = df['Neighbourhood'][indices]
        df['Neighbourhood'][indices[0]] = ', '.join(neighbour_names)
        df.drop(indices[1:], inplace=True)


# step 1
url_to_parse = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url_to_parse).text

# step 2
parsed = BeautifulSoup(html)
table_html = str(parsed.table)

# step 3
df = pd.read_html(table_html)[0]

# step 4
df.replace({'Borough' : {'Not assigned' : np.nan}}, inplace=True)
df.dropna(inplace=True)

# step 5
replaced = np.where(
    df['Neighbourhood'] == 'not assigned',  # the condition
    df['Borough'],  # choose value from array if condition is True
    df['Neighbourhood'],  # choose value from array if condition is False
)
df['Neighbourhood'] = replaced

# step 6
merge_duplicate(df)

# step 7
df.reset_index(inplace=True, drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Append Geographic Data

## Download the data

shame on Google

In [3]:
geo_data_url = 'https://cocl.us/Geospatial_data'
df_geo = pd.read_csv(geo_data_url)

## Reorder the new data frame

the rows in the `df_geo` table is re-ordered to match the `df`

In [4]:
a0 = np.array(df['Postal Code'])
a1 = np.array(df_geo['Postal Code'])
new_indices = [np.where(a1 == val)[0][0] for val in a0]
df_geo_ordered = df_geo.reindex(new_indices)
df_geo_ordered.reset_index(drop=True)
df_geo_ordered.head()

,Postal Code,Latitude,Longitude
25,M3A,43.753259,-79.329656
34,M4A,43.725882,-79.315572
53,M5A,43.654260,-79.360636
71,M6A,43.718518,-79.464763
85,M7A,43.662301,-79.389494


## Concatenate the latitude and longitude

In [5]:
if 'Latitude' not in df.columns:  # only concatenate once
    df = pd.concat([ df, df_geo_ordered ], axis=1, ignore_index=False)
df.head()

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
